# Building a Representative Corpus of Jazz Piano Trio Recordings

Procedure:
1. Authenticate with both LastFM and Musicbrainz APIs;
2. Scrape LastFM to get a list of the most popular jazz artists with 'trio' in their name;
3. Cross-reference the artists scraped from LastFM with two reference jazz discographies, removing those who don't feature in either;
4. Scrape Musicbrainz using the artist names and gather information on each of their releases;
5. Using the release information, group find the number of tracks recorded by all combinations of musicians that recorded under each act name;
6. Find the top 30 combinations of musicians who recorded the greatest number of tracks.

## Import dependencies, set constants etc.

In [ ]:
import pandas as pd
import numpy as np
import dotenv
import requests
import hashlib
import os
import logging
import matplotlib.pyplot as plt
import seaborn as sns
import musicbrainzngs
import warnings
from ast import literal_eval
from builtins import any as b_any
from datetime import timedelta, datetime
from dateutil.parser import parse, ParserError
from tqdm.notebook import tqdm
from requests.exceptions import ConnectionError
import csv
from collections import defaultdict

from src import utils
from src.visualise.corpus_plots import *

In [ ]:
# These options just make the outputs from each cell look cleaner
pd.set_option('display.max_rows', 20)
pd.options.mode.chained_assignment = None
logging.getLogger().setLevel(logging.ERROR)
plt.rcParams.update({'font.size': 18})
plt.style.use('default')

In [ ]:
# These variables are constants and can be changed if needed
N_PAGES = 500  # The number of LastFM pages to scrape artist information from
N_ARTISTS = 500  # The number of artists per LastFM page
LASTFM_ENDPOINT = "https://ws.audioscrobbler.com/2.0"  # This is the LastFM API root directory that we will make our calls to
GENRE = 'jazz'  # The LastFM genre to scrape for artist names
SEARCH_TAG = 'trio'  # Only consider artists that have this tag in their name
USABLE_TRACKS_WORST_CASE = 1/4    # In the worst case, we assume that only this percentage of total tracks will be usable in the corpus

In [ ]:
# Define the reader function to load in the required text file
reader = lambda f: open(fr'{utils.get_project_root()}\references\corpus_construction\{f}.txt', "r").read().splitlines()

## Authenticate APIs

To access the LastFM API, we need an API key, which must be applied for manually. The procedure for doing so is out of scope for this notebook. Follow the advice given in the [LastFM API documentation](https://www.last.fm/api/authentication): you will first need to register for a user account before you can apply for an API key.

Once you have an API key, this should be stored as an environment variables within a file called `.env` that is saved in the root directory of this project folder. Define the LastFM key as `LASTFM_API_KEY`. This will then be loaded in automatically using the `dotenv` package within the following few lines.

Musicbrainz does not currently require authentication, beyond setting a user agent. You should store a contact email in `.env`, under the `CONTACT_EMAIL` variable.

In [ ]:
# Load in the environment variables we require for authentication
dotenv.load_dotenv(rf"{utils.get_project_root()}\.env")

In [ ]:
# Load in our LastFM API key from .env
LASTFM_API_KEY = os.getenv('LASTFM_API_KEY')

In [ ]:
# This is all the authentication required for Musicbrainz
CONTACT_EMAIL = os.getenv('CONTACT_EMAIL')
LISTENBRAINZ_URL = 'https://listenbrainz.org/player/?recording_mbids='
musicbrainzngs.set_useragent('Jazz-Corpus-Analysis', 1, CONTACT_EMAIL)

In [ ]:
# These authentication variables aren't currently used, but are created now in case we do end up using them later
LASTFM_SHARED_SECRET = os.getenv('LASTFM_SHARED_SECRET')
LASTFM_TOKEN = eval(
    requests.get(f"{LASTFM_ENDPOINT}/?method=auth.gettoken&api_key={LASTFM_API_KEY}&format=json").text
)['token']
LASTFM_API_SIGNATURE = hashlib.md5(
    f"api_key{LASTFM_API_KEY}methodauth.getSessiontoken{LASTFM_TOKEN}{LASTFM_SHARED_SECRET}".encode('utf-8')
)

## Scrape artist data from LastFM using API

The following cells scrape data from the LastFM API using the information and keys created in the cells above.

In [ ]:
def get_total_streams(mbid: str) -> dict:
    """Gets the total number of streams for an artist by their ID, across all tracks and albums"""
    # Make the API request
    request = requests.get(
        f"{LASTFM_ENDPOINT}/?method=artist.getinfo&mbid={mbid}&api_key={LASTFM_API_KEY}&format=json"
    ).json()
    # Get the required variable from the request JSON object
    return {'playcount': request['artist']['stats']['playcount']}


def get_top_n_track_streams(mbid: str, num_tracks: int = 3) -> dict:
    """Gets the number of streams for the top n tracks by an artist, by their ID"""
    # Make the API request
    request = requests.get(
        f"{LASTFM_ENDPOINT}/?method=artist.gettoptracks&mbid={mbid}&api_key={LASTFM_API_KEY}&format=json"
    ).json()
    res_ = {}
    # Iterate through the required number of tracks
    for tr in range(num_tracks):
        # Get the name and play count from the given track
        res_[f'track_{tr + 1}_name'] = request['toptracks']['track'][tr]['name']
        res_[f'track_{tr + 1}_plays'] = request['toptracks']['track'][tr]['playcount']
    return res_

In [ ]:
def process_artist(artist_: dict) -> dict:
    """Central function for processing data from a single artist scraped from LastFM"""
    if SEARCH_TAG.upper() in str(artist_['name']).upper():
        # We make everything upper case to address any case-sensitivity issues
        return {
            **artist_,
            **get_total_streams(mbid=artist_['mbid']),
            **get_top_n_track_streams(mbid=artist_['mbid']),
        }

In [ ]:
# Empty list to hold our processed data
trios = []

In [ ]:
# Iterate through each page of artists on LastFM
for i in tqdm(range(1, N_PAGES)):
    # Make the API request
    try:
        tag = requests.get(
            f"{LASTFM_ENDPOINT}/?method=tag.gettopartists&tag={GENRE}&api_key={LASTFM_API_KEY}&page={i}&perpage={N_ARTISTS}&format=json"
        )
    # If we time out, log and then continue to the next page
    except ConnectionError as e:
        print(e)
        pass
    # If we receive data, iterate through each artist in turn and process it
    else:
        for k, v in tag.json().items():
            for k_, v_ in v.items():
                for lastfm_artist in v_:
                    # Try and process the data for one artist
                    try:
                        trios.append(process_artist(lastfm_artist))
                    # Catch errors resulting from irregular item construction, timeouts, and continue
                    except (TypeError, KeyError, ConnectionError) as e:
                        pass

In [ ]:
# Concatenate the artist information and drop any duplicates (by ID)
df = (
    pd.DataFrame([t for t in trios if t is not None])
    .drop_duplicates(subset=['mbid'])
    .drop(columns=['image', 'streamable', '@attr'])
    .reset_index(drop=True)
)
for col in ['track_1_plays', 'track_2_plays', 'track_3_plays']:
    df[col] = df[col].astype(int)
df['combined_plays'] = df['track_1_plays'] + df['track_2_plays'] + df['track_3_plays']

In [ ]:
df.to_csv(fr'{utils.get_project_root()}\references\corpus_construction\lastfm_piano_trio_search_raw.csv')

In [ ]:
df = pd.read_csv(fr'{utils.get_project_root()}\references\corpus_construction\lastfm_piano_trio_search_raw.csv')

In [ ]:
BarPlotLastFMStreams(df).create_plot()

## Subset data to only include artists in reference discographies

In order to ensure that the artists we include in the corpus played a non-trivial role in jazz history, we cross-reference the list of artists scraped from LastFM with two reference discographies, defined as:

- Gioia, T. (2011). The History of Jazz (2nd ed.). New York: Oxford University Press. (`gioia`)
- Levine, M. (2011). The Jazz Theory Book. Sebastopol: Sher Music Company. (`levine`)

If an artist can be found in the dataset scraped from LastFM AND one of the following discographies AND it passes a final manual check against the inclusion criteria, then they can be included in the corpus.

In [ ]:
# Load in the reference discographies and list of appropriate artists
levine = reader('levine')
gioia = reader('gioia')
appropriate = reader('appropriate')

In [ ]:
# Subset the dataframe to include artists in one of the two discographies & that are acceptable to the inclusion criteria
df_cut = df[(df['name'].isin(levine)) | (df['name'].isin(gioia))]
df_cut = df[(df['name'].isin(appropriate))].reset_index(drop=True)

In [ ]:
df_cut.to_csv(fr'{utils.get_project_root()}\references\corpus_construction\lastfm_piano_trio_search.csv')

In [ ]:
try:
    df_cut
except NameError:
    df_cut = pd.read_csv(fr'{utils.get_project_root()}\references\corpus_construction\lastfm_piano_trio_search.csv')

In [ ]:
df_cut

## Scrape release metadata from Musicbrainz

Once we have our list of artists from LastFM, we can now turn to scraping [Musicbrainz](https://musicbrainz.org/) to gather information on their individual releases. This will enable us to understand how many trio releases individual bandleaders made, and how many tracks were on each of these releases. This will also allow us to scrape information about the individual performers on each record, which can then be used to estimate the different networks of musicians that formed part of a named 'trio' over time.

In [ ]:
def get_mbz_artist_id(artist_str):
    """Searches Musicbrainz using an artist's name and returns the ID with the closest match"""
    return musicbrainzngs.search_artists(artist=artist_str)['artist-list'][0]['id']

def get_mbz_artist_release_groups(
        artist_id_,
        release_types: tuple = ('album', 'live'),
        limit: int = 100,
):
    """Returns all release groups associated with a particular artist, by their musicbrainz ID.

    A release group is, essentially, a 'master' recording that may have been printed and pressed under
    multiple different titles, in different formats, and in different countries over the years.
    """
    for release_type in release_types:
        # This gets the item offset
        offset = int(np.floor(musicbrainzngs.browse_release_groups(
            artist=artist_id_,
            release_type=release_type,
            limit=1,
        )['release-group-count'] / 100))
        for off in range(offset + 1):
            yield from musicbrainzngs.browse_release_groups(
                artist=artist_id_,
                release_type=release_type,
                limit=limit,
                includes=['artist-credits'],
                offset=off
            )['release-group-list']


def get_earliest_mbz_release_from_release_group(
        release_grp_: dict,
        release_status: tuple = 'official',
        limit: int = 100,
) -> dict | None:
    """Get the information relating to the first release of a record from a release group"""

    # Browse musicbrainz to get all releases under this release group
    releases_ = musicbrainzngs.browse_releases(
        release_group=release_grp_['id'],
        release_status=release_status,
        limit=limit,    # We don't use offsets here, we won't have more than 100 unique releases of one record
    )['release-list']
    # Empty lists to store our release data and ID variables
    dates = []
    ids = []
    # Iterate through all the releases of this record
    for rel in releases_:
        # Try and parse the date to a machine-readable format, and keep the ID too
        try:
            dates.append(parse(rel['date'], fuzzy=True))
        # If we don't have a date for this release, continue to the next one
        except KeyError:
            continue
        # Keep the IDs of any releases where we also have dates
        else:
            ids.append(rel['id'])
    # Return the release whose ID matches the ID of the chronologically-earliest release
    try:
        return [rel for rel in releases_ if rel['id'] == ids[np.argmin(dates)]][0]
    # If none of the releases have dates associated with them, try and return the first release in the list
    except (ValueError, IndexError):
        try:
            return releases_[0]
        # If we have an error here, it's because we don't have any releases in the first place, so return None
        except IndexError:
            return None


def getter(collection: str, attribute: str, nest: str = None, fallback=''):
    """Simple wrapper function for attempting to get an item from a dictionary"""
    try:
        # If we've passed a nest level in, subset our collection by this level
        if nest is not None:
            collection = collection[nest]
        return collection[attribute]
    # If we don't have our attribute in the dictionary, return our fallback (empty string)
    except KeyError:
        return fallback


def get_work_info(track):
    try:
        work_list = track['recording']['work-relation-list']
    except KeyError:
        return {}
    else:
        for work in work_list:
            yield {
                'work_begin': getter(work, 'begin'),
                'work_end': getter(work, 'end'),
                'work_title': getter(work, 'title', nest='work'),
                'work_type': getter(work, 'type'),
            }


def get_release_info(rel):
    return {
        'release_title': getter(rel, 'title', nest='release'),
        'release_status': getter(rel, 'status', nest='release'),
        'release_quality': getter(rel, 'quality', nest='release'),
        'release_id': getter(rel, 'id', nest='release'),
        'release_packaging': getter(rel, 'packaging', nest='release'),
        'release_date': getter(rel, 'date', nest='release'),
        'release_country': getter(rel, 'country', nest='release'),
    }

def get_artist_info(track):
    try:
        artist_list = track['recording']['artist-relation-list']
    except KeyError:
        return {}
    else:
        for artist_ in artist_list:
            yield {
                'artist_id': getter(artist_, 'id', nest='artist'),
                'artist_role': getter(artist_, 'attribute-list'),
                'artist_name': getter(artist_, 'name', nest='artist'),
                'artist_type': getter(artist_, 'type'),
                'artist_begin': getter(artist_, 'begin'),
                'artist_end': getter(artist_, 'end'),
                'artist_details': getter(artist_, 'disambiguation', nest='artist'),
            }

def get_release_artist(track):
    try:
        return [getter(a['artist'], 'name') for a in track['recording']['artist-credit'] if isinstance(a, dict)]
    except (KeyError, TypeError):
        return ''


def get_recording_info(track):
    return {
        'recording_position': getter(track, 'position'),
        'recording_number': getter(track, 'number'),
        'recording_length': getter(track, 'length'),
        'recording_id_for_lbz': getter(track, 'id', nest='recording'),
        'recording_title': getter(track, 'title', nest='recording'),
        'track_id_maybe_not_needed': getter(track, 'id')
    }


def parse_mbz_release_info(release: dict, searched_artist_name: str) -> dict:
    """Parses artist and contributor info from a musicbrainz release"""
    # Get the release by our ID and include the values we need
    rel = musicbrainzngs.get_release_by_id(
        id=release['id'],
        includes=[
            'artist-credits',
            'labels',
            'recordings',
            'recording-level-rels',
            'work-rels',
            'work-level-rels',
            'artist-rels'
        ],
        release_status=['official']
    )
    release_info = get_release_info(rel)
    # Medium list: the list of 'things' in a release, e.g. individual CDs, discs
    try:
        medium_list = rel['release']['medium-list']
    except KeyError:
        return
    else:
        for medium in medium_list:
            # Track list: the list of pieces/songs/tracks on a medium
            try:
                track_list = medium['track-list']
            except KeyError:
                continue
            else:
                for track in track_list:
                    yield {
                        'searched_artist_name': searched_artist_name,
                        'release_artist': get_release_artist(track),
                        **release_info,
                        **get_recording_info(track),
                        'artist_list': list(get_artist_info(track)),
                        'work_list': list(get_work_info(track)),
                    }


def get_artist_title_for_writing(name):
    """Replaces invalid characters in an artist name string so that it can be saved"""
    return name.replace(" ", "_").replace("\\", "_").replace('/', '_').lower()

def write_artist_release_info_to_csv(artist_releases: list, artist_name: str):
    """Dumps release information from a particular artist to a .csv file"""
    pd.DataFrame(artist_releases).to_csv(fr'{utils.get_project_root()}\references\corpus_construction\musicbrainz_search\{artist_name}.csv')


# Obtain the list of artists that we've already processed
preprocessed = os.listdir(fr'{utils.get_project_root()}\references\corpus_construction\musicbrainz_search')
# Read in the list of valid artist names from our text file
musicbrainz_artists = pd.read_csv(fr'{utils.get_project_root()}\references\corpus_construction\musicbrainz_artists.csv')['artist'].to_list()
# Iterate over every artist (with a progress bar)
for artist in tqdm(musicbrainz_artists):
    # Skip processing the artist if we've already done so in the past
    if f"{get_artist_title_for_writing(artist)}.csv" in preprocessed:
        continue
    res = []
    # Get the ID of our artist from Musicbrainz
    artist_id = get_mbz_artist_id(artist)
    # Get all release groups under that artist ID
    release_grps = list(get_mbz_artist_release_groups(artist_id))
    for release_grp in tqdm(release_grps, desc=f"{artist}, MBID: {artist_id.upper()}"):
        # Get the earliest release for that release group
        earliest_release = get_earliest_mbz_release_from_release_group(release_grp)
        if earliest_release is not None:
            # Parse artist info from the earliest release
            release_info = list(parse_mbz_release_info(earliest_release, artist))
            # Extend the list we're creating
            res.extend(release_info)
    # Once we've scraped all the releases for an artist, we can dump the information to a csv.
    write_artist_release_info_to_csv(res, get_artist_title_for_writing(artist))

## Clean Musicbrainz output

### Load the musicbrainz search results

For every bandleader obtained from the LastFM search, we dumped `.csv` files from scraping Musicbrainz. Now, we'll load these all in as a single `pd.DataFrame` object.

In [ ]:
def process_release_with_multiple_works(work_list):
    """For recordings associated with multiple works, e.g. medleys, process data to get one result for the recording"""
    works = defaultdict(set)
    for d in work_list:
        for k, v in d.items():
            works[k].add(v)
    works = dict(works)
    for k, v in works.items():
        try:
            works[k] = [i for i in works[k] if i != ''][0]
        except IndexError:
            works[k] = ''
    return works

In [ ]:
rows = []
search_dir = fr"{utils.get_project_root()}\references\corpus_construction\musicbrainz_search"
for file in tqdm(os.listdir(search_dir)):
    with open(search_dir + '\\' + file, encoding='utf8') as f:
        for row in csv.DictReader(f, skipinitialspace=True):
            work_list = literal_eval(row['work_list'])
            if len(work_list) == 0:
                works = {col: '' for col in ['work_begin', 'work_end', 'work_title', 'work_type']}
            elif len(work_list) == 1:
                works = work_list[0]
            elif len(work_list) > 1:
                works = process_release_with_multiple_works(work_list)
            art_list = literal_eval(row['artist_list'])
            row.pop('artist_list', None)
            row.pop('work_list', None)
            for art in art_list:
                rows.append(row | art | works)

mbz = pd.DataFrame(rows).reset_index(drop=True)

### Get the estimated date of recording

This is complicated. Musicbrainz gives us four dates we can readily access, relating to when a recording was made. These are called `artist_begin`, `artist_end`, `work_begin`, and `work_end`. In many cases, these will all be the same. In some situations, such as when a recording was made over multiple days, however, we will get a range of dates, corresponding to the first and last date of recording. What we can do here is interpolate between the 25th and 75th quantile date in these range, to estimate the day of recording.

We use quantiles, rather than first/last date, to account for issues where a date may have been entered incorrectly (e.g. [`Return to Forever`](https://musicbrainz.org/recording/64a92f98-fd34-4346-8a31-295858b3dbb6), Stanley Clarke's `artist_begin` is `1772`, rather than `1972` as it is for the other performers)

In [ ]:
# For any release, set missing dates equal to the most frequently observed date for that release
mbz_ = mbz.copy(deep=True)
for idx, grp in mbz_.groupby('release_id'):
    # Get all dates as one list
    sub = grp[['artist_begin', 'artist_end', 'work_begin', 'work_end']].melt().drop('variable',axis=1)['value']
    # Get non-missing dates
    non_zero = sub[sub != '']
    # Try and get the most commonly observed date, or use NaN if all dates are missing
    mode = np.nan
    try:
        mode = non_zero.mode().iloc[0]
    except IndexError:
        mode = np.nan
    finally:
        # Set missing dates equal to the mode/nan
        for col in ['artist_begin', 'artist_end', 'work_begin', 'work_end']:
            mbz_.loc[grp[grp[col] == ''][col].index, col] = mode

In [ ]:
# Correct date columns to datetime object
def parser(x):
    if isinstance(x, float):
        x = np.floor(x)
    try:
        return parse(str(x), fuzzy=True, default=datetime(3000, 1, 1, 0, 0))
    except ParserError:
        return pd.NaT


for col in ['release_date', 'artist_begin', 'artist_end', 'work_begin', 'work_end']:
    mbz_[col] = mbz_[col].apply(parser)

In [ ]:
# For all of our available dates, interpolate between the 25th and 75th quantile date to estimate the day of recording
def interpolate_date(grp) -> datetime:
    """Interpolate between an array of datetime objects"""
    melt = grp[['artist_begin', 'artist_end', 'work_begin', 'work_end']].melt().drop('variable',axis=1)['value']
    begin = melt.quantile(0.25).to_pydatetime()
    end = melt.quantile(0.75).to_pydatetime()
    try:
        diff = timedelta(seconds=(end - begin).total_seconds() / 2)
        grp['recording_date_estimate'] = begin + diff
    except ValueError:
        grp['recording_date_estimate'] = pd.NaT
    return grp['recording_date_estimate']


mbz_['recording_date_estimate'] = mbz_.groupby('release_id', group_keys=False).apply(interpolate_date)

### Clean artist type and role variables

In [ ]:
# Transfer information on performer type to role column
for ty in ['vocal', 'performing orchestra', 'samples from artist', 'misc']:
    mbz_.loc[mbz_['artist_type'] == ty, 'artist_role'] = f"['{ty}']"
    mbz_.loc[mbz_['artist_type'] == ty, 'artist_type'] = 'instrument'

In [ ]:
# Convert the string representation of a list column to an actual Python list, catching invalid rows
mbz_['artist_role_'] = mbz_['artist_role'].apply(lambda ro: literal_eval(str(ro)) if str(ro) != '' else '')

### Drop non-performer contributor types
These are mostly producers, mixing engineers, etc. who are credited on releases but did not actually perform on them

In [ ]:
mbz_ = mbz_[mbz_['artist_type'].isin(['instrument', 'performer'])]

### Drop tracks containing invalid instruments

When we scraped Musicbrainz, we collected all recordings made by a particular bandleader: this includes their trio recordings, but also recordings made in any other configuration, such as quartets, quintets, big band, solo recordings, etc. We need to remove these by dropping any tracks where an instrument that is not part of the standard piano-bass-drums lineup was reported to have performed.

In [ ]:
# We need at least three artists to be credited as playing these instruments
include_instrs = [
    'bass', 'drums', 'piano',
]
# Disqualify a release if it contains these instruments
# TODO: can we create this list iteratively?
exclude_instrs = [
    'French horn', 'woodwind', 'solo', 'reeds', 'membranophone', 'oboe', 'harmonica', 'flugelhorn', 'gumbri', 'piccolo', 'bell', 'harp', 'congas', 'bongos', 'mellophone', 'accordion', 'tabla', 'washtub bass', 'tambourine', 'clavinet', 'additional', 'chamberlin', 'banjo', 'marimba', 'baritone', 'brass', 'saxophone', 'clarinet', 'trombone', 'guitar', 'strings', 'cello', 'viol', 'vibraphone', 'electric', 'flute', 'tuba', 'oon', 'percussion', 'foot stomps', 'bass drum', 'celesta', 'tack', 'keyboard', 'Rhodes', 'timpani', 'duo', 'organ', 'electronic', 'synthesizer', 'vocal', 'executive', 'co', 'recorder', 'vocoder', 'slit drum', 'family', 'performing orchestra', 'samples from artist', 'misc'
]

In [ ]:
def check_instrument(row: pd.Series) -> bool:
    """Returns True if an artist performed a valid instrument and did not perform an invalid instrument"""
    try:
        return all([
            any([b_any(word.lower() in x.lower() for x in row) for word in include_instrs]),
            not any([b_any(word.lower() in x.lower() for x in row) for word in exclude_instrs])
        ])
    except TypeError:
        return False

# TODO: we can probably simplify this
mbz_['include_release'] = mbz_['artist_role_'].apply(check_instrument)
# Drop tracks containing at least one invalid instrument
mbz_ = (
    mbz_.groupby(['recording_title', 'recording_title', 'recording_id_for_lbz'])
        .filter(lambda x: not any(x['include_release'] == False))
)

In [ ]:
# Assert that all tracks only contain valid instruments
assert all(
    [len(grp[grp['include_release'] == False]) == 0 for idx, grp in mbz_.groupby(['recording_title', 'release_title', 'recording_id_for_lbz'])]
)

### Drop duplicates of one recording

In [ ]:
mbz_ = mbz_.drop_duplicates(subset=['recording_title', 'recording_length', 'release_title', 'recording_id_for_lbz', 'artist_name'], keep='first')

### Format instrument names

The names of instruments returned from Musicbrainz aren't standardized: for example, `piano` may be referred to using `piano`, `grand piano`, `piano (solo)`, among others. We can format these fairly easily.

In [ ]:
def format_instrument(row: pd.Series):
    checks = [b_any(word in x for x in row) for word in include_instrs]
    return ''.join(np.array(include_instrs)[np.where(checks)[0]])

mbz_['artist_instrument'] = mbz_['artist_role_'].apply(format_instrument)

### Drop recordings containing more than one performer on a single instrument

These are recordings where more than one performer played on a single instrument in the trio configuration. These usually have fairly obvious-to-spot titles, for instance `An Evening with Herbie Hancock & Chick Corea` (contains both Herbie & Chick on piano), `Ray Brown's New Two Bass Hits` (contains both Ray Brown and Pierre Boussaguet on double bass). We can remove these by simply dropping any tracks where we have more than one unique credit for a particular trio instrument.

In [ ]:
mbz_ = (
    mbz_.groupby(['recording_title', 'recording_length', 'release_title', 'recording_id_for_lbz',])
        .filter(lambda x: not any(len(g) > 1 for _, g in x.groupby('artist_instrument')))
)

### Drop recordings that do not contain a full trio

It's quite common on an album to have occasional tracks that may only include a subset of the overall trio -- e.g. piano solos, duets between pianist and bassist. For examples, see both versions of `Epilogue` on `Everybody Digs Bill Evans`, by the `Bill Evans Trio`. We need to remove these.

In [ ]:
mbz_ = (
    mbz_.groupby(['recording_title', 'recording_length', 'release_title', 'recording_id_for_lbz',])
        .filter(lambda x: sorted(x['artist_instrument'].unique()) == sorted(include_instrs))
)

### Pivot the dataframe to get the whole ensemble on one row

The tracks that are left in the dataframe should contain one pianist, one bassist, and one drummer only. We can now pivot the dataframe to get all of these performers on the same row, making it easier to work with combinations of performers.

In [ ]:
piv = mbz_.pivot_table(
    index=['searched_artist_name', 'recording_title', 'release_title', 'recording_length', 'recording_id_for_lbz', 'release_artist', 'release_date', 'release_country', 'work_begin', 'work_end', 'recording_position', 'recording_date_estimate'],
    columns=['artist_instrument'],
    aggfunc='first',
    values='artist_name'
).reset_index(drop=False)

### Drop recordings not made by the leader

Occasionally, we can run into issues where the artist we initially searched for is not present on a track. For an example, see the album [`Just One of Those Things`](https://musicbrainz.org/release/b1f218be-3aaf-4b7c-a9ca-fab91c29f82e). We searched for `Bud Powell` to find this release, but some of the tracks on the album are by the `Duke Jordan Trio`, who we did not search for. We need to drop the tracks on this album where `Bud Powell` is not recorded as playing either piano, bass or drums.

In [ ]:
def filter_non_leader_recordings(row: pd.Series):
    """Filters tracks where the searched for leader does not appear as a performer on a track"""
    # We accomplish the filtering by looking for overlaps of at least two words between the artist name we searched for and one of the artists in the trio.
    # This means that cases where we searched for 'The Bill Evans Trio' will pass, provided that Bill Evans can be found on either piano, bass, or drums.
    return any(len(set(row['searched_artist_name'].split()) & set(art.split())) >= 2 for art in row[['bass', 'drums', 'piano']].to_list())

piv_ = piv[piv.apply(filter_non_leader_recordings, axis=1)]

### Drop incomplete and exceptionally short tracks

Occasionally albums contain tracks which are very short. This is especially common on live albums, which may include tracks variously containing words like `Introduction`, `Warm-up`, `Announcement`, etc. These usually don't actually contain any playing, and just consist of the venue MC announcing the name of the artist and the audience applauding them. In other cases, compilation albums may include tracks labelled as `Breakdown`, `False Start`, `Fragment`, etc. These are takes that don't contain a full performance and instead break down, often after only a few bars. We can remove these tracks by first dropping tracks which contain words like `warm-up`', `false start` etc. in their name, and then setting a hard cut-off to drop any track with a duration of less than half a minute.

In [ ]:
removers = ['false', 'announ', 'introd', 'warm', 'fragment', 'final', 'break', 'epilogue']
for remover in removers:
    piv_ = piv_[~piv_['recording_title'].str.lower().str.contains(remover)]

In [ ]:
# Drop recordings that do not have track runtimes provided
piv_ = piv_[piv_['recording_length'] != '']

In [ ]:
piv_['recording_length_'] = piv_['recording_length'].astype(int).apply(lambda x: timedelta(milliseconds=x))
piv_ = piv_[piv_['recording_length_'] > timedelta(seconds=30)]

### Convert performer names to first initial/surname

The names for some performers are inconsistently labelled on Musicbrainz; for example, the bassist `Eddie Gomez` is also occasionally labelled as `Edward M. Gomez`. We can account for this by converting all performer names into a common format, this being the first initial of their first name and their surname. So, `Eddie Gomez` and `Edward M. Gomez` would both become `E. Gomez`.

In [ ]:
def initials(a):
    """Converts a list of strings of arbitrary length to their first initial"""
    if len(a) == 0:
        return ''
    return ''.join(map(lambda li: li[0] + '.', [a[0]]))

def abbreviate(s):
    """Abbreviates a name to surname, first initial"""
    l = s.split()
    return f'{l[-1]}, {initials(l[0:-1])}'

for instr in include_instrs:
    piv_[f'{instr}_'] = piv_[instr].apply(abbreviate).apply(lambda s: "".join(c for c in s if ord(c) < 128))

In [ ]:
# We can also convert track numbers to integers
piv_['recording_position'] = piv_['recording_position'].astype(int)

### Match artist names with bandleaders

In [ ]:
bls = pd.read_csv(fr'{utils.get_project_root()}\references\corpus_construction\artist_birth_death.csv')
clean = pd.merge(piv_, bls, left_on='searched_artist_name', right_on='artist')

In [ ]:
# For artists who are currently alive, replace their date of death with today
now = datetime.now().strftime('%Y-%m-%d')
clean['death'] = clean['death'].fillna(now)

In [ ]:
clean['birth'] = clean['birth'].apply(parser)
clean['death'] = clean['death'].apply(parser)

### Drop tracks by the same artist duplicated across several albums

Sometimes, as in the case of compilation albums, the same track can be duplicated across multiple releases. While there is no sure-fire way to accomodate all cases where this might have happened without dropping all tracks with duplicate names -- and potentially dropping alternate takes as a result! -- one way is to search group by tracks with the same name and combination of performers, and then drop any which have very similar durations.

In [ ]:
clean = clean.reset_index(drop=True)
for idx, grp in clean.groupby(['bandleader', 'recording_title', 'piano', 'bass', 'drums']):
    lead = grp['recording_length_'].sort_values().diff()
    if len(lead) > 1:
        clean = clean.drop(lead[lead < timedelta(seconds=3)].index)

In [ ]:
# We can also drop duplicates that have the same musicbrainz id
clean = clean.drop_duplicates(subset=['bandleader', 'recording_id_for_lbz'], keep='first')

## Remove tracks recorded after a bandleader's death/before their birth

This is usually just due to faulty record keeping on Musicbrainz: e.g. a few tracks on Bill Evans Trio, [Live At The Penthouse Seattle 1966](https://musicbrainz.org/release/f8197342-b35e-4783-b8d8-2bf542cfb51c) are listed as being recorded in 1996, rather than 1966.

In [ ]:
clean = clean[(clean['death'] > clean['recording_date_estimate']) & (clean['birth'] < clean['recording_date_estimate'])]

In [ ]:
clean[clean['piano_'] == 'Hicks, J.']

In [ ]:
clean['trio'] = clean[['piano_', 'bass_', 'drums_']].agg('/'.join, axis=1)

### Save the cleaned dataframe to a .csv to make loading easier

In [ ]:
clean.to_csv(fr'{utils.get_project_root()}\references\corpus_construction\mbz_output_cleaned.csv')

In [ ]:
clean = pd.read_csv(fr'{utils.get_project_root()}\references\corpus_construction\mbz_output_cleaned.csv', parse_dates=['recording_date_estimate', 'recording_length_'])

In [ ]:
import matplotlib as mpl
import src.visualise.visualise_utils as vutils

BarPlotBandleaderDuration(clean).create_plot()
plt.show()

### Get the names of the 10 bandleaders with the most recordings

In [ ]:
bandleaders = ['Bill Evans', 'Ahmad Jamal', 'Bud Powell', 'Oscar Peterson', 'Keith Jarrett', 'Tommy Flanagan', 'Junior Mance', 'Kenny Barron', 'John Hicks', 'McCoy Tyner']

In [ ]:
clean['recording_length_']

In [ ]:
small = clean.groupby('bandleader').agg({'recording_title': 'count', 'recording_length_': 'sum'}).reset_index(drop=False)
small['recording_length_'] = small['recording_length_'].dt.total_seconds()

In [ ]:
include = clean[clean['bandleader'].isin(bandleaders)].reset_index(drop=True)

### Plot bandleaders with most number of tracks, and with longest total length of recordings

In [ ]:
BarPlotBandleaderDuration(clean).create_plot()

## Subset included data using anchor points

In [ ]:
def get_anchorpoint(df, n_tracks = 30):
    first = df['recording_date_estimate'].min()
    last = df['recording_date_estimate'].max()
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        return [
            a.to_pydatetime() for a in pd.date_range(
                first, last, n_tracks, normalize=True, inclusive='both'
            ).to_list()
        ]

def get_anchorranges(anchors):
    start = []
    end = []
    for a1, a2, a3 in zip(anchors[:], anchors[1:], anchors[2:]):
        start.append(a2 - ((a2 - a1) / 2))
        end.append(a2 + ((a3 - a2) / 2))
    start.insert(0, anchors[0])
    end.insert(0, anchors[0] + (anchors[1] - anchors[0]) / 2)
    start.append(anchors[-1] - (anchors[-1] - anchors[-2]) / 2)
    end.append(anchors[-1] + timedelta(days=1))
    return [(s, a, e) for s, a, e in zip(start, anchors, end)]


def subset_df_by_anchor(grp: pd.DataFrame, anc_range: tuple):
    cols = [
        'anchor_number',
        'anchor_point',
        'recording_date_estimate',
        'days_from_anchor',
        'bandleader',
        'piano',
        'bass',
        'drums',
        'recording_title',
        'recording_position',
        'release_title',
        'recording_length_',
        'recording_id_for_lbz'
    ]
    for num, (start, anc, end) in enumerate(anc_range, 1):
        subset = grp[(grp['recording_date_estimate'] >= start) & (grp['recording_date_estimate'] < end)]
        subset['recording_length_'] = subset["recording_length_"].astype(str).str.replace('0 days 00:', '')
        subset['recording_id_for_lbz'] = LISTENBRAINZ_URL + subset['recording_id_for_lbz']
        subset['anchor_number'] = num
        subset['anchor_point'] = anc.strftime('%Y-%m-%d')
        subset['days_from_anchor'] = (subset['recording_date_estimate'] - anc).abs().dt.days
        yield (
            subset[cols]
                  .sort_values(['days_from_anchor', 'release_title', 'recording_position'])
                  .reset_index(drop=True)
        )

In [ ]:
include

## Write subsetted data

In [ ]:
df_dict = {}
for idx, grp in include.groupby('bandleader'):
    anchors = get_anchorpoint(grp)
    anchorranges = get_anchorranges(anchors)
    subs = pd.concat(subset_df_by_anchor(grp, anchorranges)).reset_index(drop=True)
    assert len(subs) == len(grp)
    df_dict[idx] = subs

In [ ]:
fp = fr'{utils.get_project_root()}\references\corpus_construction\anchor_search_300_slices_15.xlsx'
with pd.ExcelWriter(fp, engine='openpyxl') as writer:
    for key, df in df_dict.items():
        df.to_excel(writer, sheet_name=f'{key}', index=False)

In [ ]:
include.to_csv(fr'{utils.get_project_root()}\\references\corpus_construction\include_recordings.csv')

## Subset data for Bill Evans case study

In [ ]:
include = pd.read_csv(fr'{utils.get_project_root()}\\references\corpus_construction\include_recordings.csv', parse_dates=['recording_date_estimate', 'birth', 'death'])

In [ ]:
def evans_fmt(subset):
    cols = [
        'recording_title',
        'release_title',
        'recording_date_estimate',
        'bass',
        'drums',
        'recording_position',
        'recording_length_',
        'recording_id_for_lbz'
    ]
    subset['recording_length_'] = subset["recording_length_"].astype(str).str.replace('0 days 00:', '')
    subset['recording_id_for_lbz'] = LISTENBRAINZ_URL + subset['recording_id_for_lbz']
    return subset[cols].sort_values(by=['release_title', 'recording_position'])

In [ ]:
fp = fr'{utils.get_project_root()}\references\corpus_construction\trio_search_evans.xlsx'
include_trios = include[include['bandleader'] == 'Bill Evans'].groupby('trio').agg({'recording_length': 'sum'}).sort_values('recording_length', ascending=False).head(7).reset_index()['trio'].values
with pd.ExcelWriter(fp, engine='openpyxl') as writer:
    for idx, trio in include[include['trio'].isin(include_trios)].groupby('trio'):
        evans_fmt(trio).to_excel(writer, sheet_name=idx.replace('Evans, B./', '').replace('/', '-'), index=False)

In [ ]:
include['recording_length_'] = include['recording_length'].astype(int).apply(lambda x: timedelta(milliseconds=x)).dt.total_seconds()
include_trios = include[include['bandleader'] == 'Bill Evans'].groupby('trio').agg({'recording_length': 'sum'}).sort_values('recording_length', ascending=False).head(7).reset_index()['trio'].values
small = include[include['bandleader'] == 'Bill Evans'].groupby('trio').agg({'recording_length_': 'sum'}).reset_index(drop=False)
small['hours'] = small['recording_length_'] / 3600
small.sort_values(by='hours', ascending=False)

Seven trios recorded at least 5.5 hours of material: all the remaining trios recorded less than 2.5 hours. These latter trios were likely recruited for a specific concert or recording date (or to subsitute in for another, more permanent musician), and do not necessarily constitute a working line-up of 'The Bill Evans Trio'

In [ ]:
d = small.sort_values(by='hours', ascending=False)
d['trio'] = d['trio'].str.replace('Evans, B./', '')
d['include_'] = d['hours'] > 5
g = sns.barplot(data=d, x='trio', y='hours', hue='include_', dodge=False)
g.set_xticks(g.get_xticks(), g.get_xticklabels(), rotation=45, ha='right')
g.set(xlabel='Trio', ylabel='Total hours')
g.get_legend().set_title('Include?')

## Visualize the results!

### Visualise recording year

In [ ]:
# Load the corpus to get acceptable tracks
corp = pd.DataFrame(utils.CorpusMaker.from_excel('corpus_chronology', keep_all_tracks=False).tracks)
tracks_in_corp = corp['mbz_id'].to_list()
include['in_corpus'] = include['recording_id_for_lbz'].isin(tracks_in_corp)

In [ ]:
TimelinePlotBandleaders(include, include_images=False).create_plot()

In [ ]:
include['duration'] = pd.to_timedelta(include['recording_length_']).dt.total_seconds()

BoxPlotRecordingLength(include).create_plot()
plt.show()

In [ ]:
BarPlotCorpusDuration(include, include_images=False).create_plot()
plt.show()

## Network visualisation

In [ ]:
import networkx as nx
import pandas as pd

G = nx.from_pandas_edgelist(
    pd.DataFrame({
        'source': pd.concat((include['piano_'], include['piano_'], include['bass_'], include['bass_'], include['drums_'], include['drums_'])),
        'target': pd.concat((include['bass_'], include['drums_'], include['piano_'], include['drums_'], include['bass_'], include['piano_']))
    }),
)

In [ ]:
from pyvis.network import Network
import src.visualise.visualise_utils as vutils
import re

nt = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", filter_menu=True, cdn_resources='remote')
nt.repulsion()
# populates the nodes and edges data structures
nt.from_nx(G)
neighbor_map = nt.get_adj_list()


def _get_instr(neigh):
    res = []
    for n in neigh:
        if n in include['piano_'].values:
            res.append(n + ' (P)')
        elif n in include['bass_'].values:
            res.append(n + ' (B)')
        else:
            res.append(n + ' (D)')
    return sorted(res)

# add neighbor data to node hover data
for node in nt.nodes:
    if node['label'] in include['piano_'].values:
        node['color'] = vutils.RED
    elif node['label'] in include['bass_'].values:
        node['color'] = vutils.GREEN
    else:
        node['color'] = vutils.BLUE


    node["title"] = "Recorded with: \n" + "\n".join(_get_instr(neighbor_map[node["id"]]))
    node["value"] = len(neighbor_map[node["id"]])

# nt.show(rf'{utils.get_project_root()}\reports\figures\corpus_plots\trio_network_visualisation.html', notebook=False)
nt.show('trio_network_visualisation.html', notebook=False)

In [ ]:
inc = include[include['in_corpus']]
include.groupby('drums_')['bandleader'].nunique().sort_values()

In [ ]:
include.groupby(['bass_', 'drums_'])['bandleader'].count().sort_values()

In [ ]:
corp['time_signature'].value_counts()